In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
# hyperparameters
batch_size = 64 # how many independent sequences will we process in parallel?
block_size = 256 # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 500
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
eval_iters = 200
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
# ------------

torch.manual_seed(1337)
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=500)[0].tolist()))
torch.save(model.state_dict(), 'model.pth')
#open('more.txt', 'w').write(decode(m.generate(context, max_new_tokens=10000)[0].tolist()))


cpu
10.788929 M parameters


KeyboardInterrupt: 

In [3]:
model = GPTLanguageModel()
model.load_state_dict(torch.load('model.pth', map_location='cpu'))
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))


ANUS:
Yo'er blent bid thee in the treaches on what;
Taking thus honour in thy face.
Cature it, loves betimes; when so oft thine
I cannot stince be in the veigh?

First Water:
The caulth in recense bearship the strange out
issing of his eye on the spirit
inforce itseluy, were puspecion; if for the posed
of presence be allowant and unmost absoluto; our
he vences are secred the father usure,
it owads so, it is a momonth any countinues the plot
damned. With thereof it is ensoil'd
our huning Virtues:


In [ ]:
import torch
import random
from torch.distributions import MultivariateNormal
'''
Collect training data activations: Forward pass normal examples through your neural network and record activations at each layer
Fit Gaussian distributions per layer: For each layer i, calculate empirical mean μᵢ and covariance matrix Σᵢ from the training activations
Create sparse argument masks: Start with diagonal covariance matrices (independence assumption) - boolean masks that select which covariance terms to track
For new inputs: Forward pass through network to get activations x*ᵢ at each layer
Compute anomaly score: Sum the negative log probabilities across layers: Σᵢ (-log Pᵢ(x*ᵢ)) where Pᵢ = N(μᵢ, Σᵢ)
Set threshold: Use validation data to determine anomaly threshold (e.g., 95th percentile of normal examples)
Flag anomalies: Inputs scoring above threshold are anomalies
'''

def calculate_layer_statistics(activations):
    stacked_activations = []
    for activation in activations:
        # Ensure consistent dtype
        activation = activation.float()
        # Flatten batch and time dimensions: (B, T, n_embd) -> (B*T, n_embd)
        flattened = activation.view(-1, activation.shape[-1])
        stacked_activations.append(flattened)
    all_samples = torch.cat(stacked_activations, dim=0)
    # empirical mean μ̂ = (1/N) Σ(i=1 to N) x_i
    empirical_mean = torch.mean(all_samples, dim=0)
    # Center the data
    centered_samples = all_samples - empirical_mean.unsqueeze(0)
    # covariance matrix Σ̂ = (1/N) Σ(i=1 to N) (x_i - μ̂)(x_i - μ̂)^T
    covariance_matrix = torch.mm(centered_samples.t(), centered_samples) / (all_samples.shape[0] - 1)
    
    return empirical_mean, covariance_matrix

def create_sparse_argument_masks(covariance_matrix):
    dim = covariance_matrix.shape[0]
    # Create diagonal mask (to assume independence between activations)
    mask = torch.eye(dim, dtype=torch.bool, device=covariance_matrix.device)
    return mask

def make_layer_mask(best_layer_mask, covariance_matrix, lr=0.05):
    if best_layer_mask is None:
        return create_sparse_argument_masks(covariance_matrix)
    
    new_mask = best_layer_mask.clone()
    dim = best_layer_mask.shape[0]
    
    def is_positive_definite(matrix):
        try:
            torch.linalg.cholesky(matrix)
            return True
        except torch.linalg.LinAlgError:
            return False
    
    # Go through upper triangular positions
    positions = [(i, j) for i in range(dim) for j in range(i, dim)]
    random.shuffle(positions)
    for i, j in positions:
        # Skip with probability (1 - lr)
        if random.random() >= lr:
            continue
        # Try flipping the bit
        original_val = new_mask[i, j].item()
        new_mask[i, j] = not original_val
        # Maintain symmetry
        if i != j:
            new_mask[j, i] = new_mask[i, j]
        # Test if the masked matrix is still positive definite
        masked_matrix = covariance_matrix * new_mask.float()
        if not is_positive_definite(masked_matrix):
            new_mask[i, j] = original_val
            if i != j:
                new_mask[j, i] = original_val
    
    return new_mask

def get_activation(input_points, layer, model, device='cpu'):
    layer_activations = []
    block_idx = layer
    for sampled_activation in input_points:
        sampled_activation = sampled_activation.to(device).float()  # Ensure float type
        with torch.no_grad():
            x = sampled_activation
            # Pass through the specific transformer block
            block = model.blocks[block_idx]
            # Pre-attention layer norm
            ln1_out = block.ln1(x)
            # Self-attention
            sa_out = block.sa(ln1_out)
            # Residual connection after attention
            x = x + sa_out
            # Pre-feedforward layer norm
            ln2_out = block.ln2(x)
            # Feedforward
            ffwd_out = block.ffwd(ln2_out)
            # Residual connection after feedforward
            x = x + ffwd_out
            layer_activations.append(x)
    return layer_activations

def propagate_distribution(input_points, layer, model):
    layer_activations = get_activation(input_points, layer, model)
    layer_empirical_mean, layer_covariance_matrix = calculate_layer_statistics(layer_activations)
    return layer_empirical_mean, layer_covariance_matrix, layer_activations

def create_gaussian_distribution(empirical_mean, covariance_matrix, layer_mask):
    # Only keep covariance terms where mask is True, set others to 0
    masked_cov = covariance_matrix * layer_mask.float()
    masked_cov = 1e-6 * torch.eye(masked_cov.shape[0])
    distribution = MultivariateNormal(loc=empirical_mean, covariance_matrix=masked_cov)
    return distribution

def sample_next_layer_inputs(estimated_layer_gaussian, original_activations, num_activations):
    # Get the original shape from one of the activations
    sample_activation = original_activations[0]
    batch_size, seq_len, feature_dim = sample_activation.shape
    
    # Sample from the distribution
    # We need (num_activations * batch_size * seq_len) samples
    total_samples_needed = num_activations * batch_size * seq_len
    sampled_flat = estimated_layer_gaussian.sample((total_samples_needed,))
    
    # Reshape to match the expected format: (num_activations, batch_size, seq_len, feature_dim)
    sampled_activations = sampled_flat.view(num_activations, batch_size, seq_len, feature_dim)
    
    # Convert back to list format expected by the rest of the code
    current_layer_inputs = [sampled_activations[i] for i in range(num_activations)]
    
    return current_layer_inputs

def find_true_outputs(model, input_points, device='cpu'):
    true_outputs = []
    generated_tokens = []  # Store the actual tokens generated
    
    with torch.no_grad():
        for input_point in input_points:
            input_point = input_point.to(device).long()  # Ensure long type for embeddings
            B, T = input_point.shape
            
            # Token embeddings
            tok_emb = model.token_embedding_table(input_point).float()  # (B, T, n_embd)
            
            # Position embeddings
            pos_emb = model.position_embedding_table(torch.arange(T, device=device)).float()  # (T, n_embd)
            
            # Combined embeddings
            x = tok_emb + pos_emb  # (B, T, n_embd)
            
            # Pass through all transformer blocks
            for block in model.blocks:
                # Pre-attention layer norm
                ln1_out = block.ln1(x)
                # Self-attention
                sa_out = block.sa(ln1_out)
                # Residual connection after attention
                x = x + sa_out
                # Pre-feedforward layer norm
                ln2_out = block.ln2(x)
                # Feedforward
                ffwd_out = block.ffwd(ln2_out)
                # Residual connection after feedforward
                x = x + ffwd_out
            
            # Final layer norm
            ln_f_out = model.ln_f(x)  # (B, T, n_embd)
            
            # Language model head (logits)
            logits = model.lm_head(ln_f_out)  # (B, T, vocab_size)
            
            # Convert logits to actual tokens
            # Take the last token prediction for each sequence
            last_token_logits = logits[:, -1, :]  # (B, vocab_size)
            predicted_token_ids = torch.argmax(last_token_logits, dim=-1)  # (B,)
            
            # Decode to actual text
            for batch_idx in range(B):
                token_id = predicted_token_ids[batch_idx].item()
                predicted_char = itos[token_id]  # Convert to character using your decoder
                generated_tokens.append(predicted_char)
            
            # Flatten the logits to match expected input format for calculate_layer_statistics
            # This converts (B, T, vocab_size) -> (B*T, vocab_size)
            flattened_logits = logits.view(-1, logits.shape[-1])
            true_outputs.append(flattened_logits)
            
    return true_outputs, generated_tokens
    
def surprise_of_estimate(true_samples, estimated_output_distribution):
    total_log_prob = 0.0
    sample_count = 0
    
    for sample in true_samples:
        # Process each sample individually
        for i in range(sample.shape[0]):
            single_sample = sample[i]  # Single vector of vocab_size
            log_prob = estimated_output_distribution.log_prob(single_sample)
            total_log_prob += log_prob
            sample_count += 1
    
    return -(total_log_prob / sample_count).item()

def surprise_of_argument_metric1(layer_masks, covariance_matrices):
    total_surprise = 0.0
    for i, (mask, cov_matrix) in enumerate(zip(layer_masks, covariance_matrices)):
        # Count number of True elements in the mask (|πᵢ|)
        num_tracked_elements = torch.sum(mask.float()).item()
        # Dimension of the space (dim(Xᵢ))
        dimension = cov_matrix.shape[0]
        # Surprise is the excess over diagonal elements
        # We subtract dimension because diagonal elements don't add surprise
        # (they represent the base independence assumption)
        layer_surprise = num_tracked_elements - dimension
        total_surprise += max(0, layer_surprise)  # Only count positive deviations
    return total_surprise

def surprise_of_argument_metric2(layer_masks, estimated_gaussians, naive_gaussians):
    total_surprise = 0.0
    for i, (estimated_dist, naive_dist) in enumerate(zip(estimated_gaussians, naive_gaussians)):
        # Compute KL divergence: KL(P_i^π || Q_i)
        kl_div = torch.distributions.kl_divergence(estimated_dist, naive_dist)
        if torch.isfinite(kl_div):
            total_surprise += kl_div.item()
        else:
            # If KL divergence is infinite/NaN, add a large penalty
            total_surprise += 1000.0
    return total_surprise

def score_input(activation, best_heuristic_argument):
    #Anomaly Score = Σᵢ (-log Pᵢᵖ(x*ᵢ))
    score = 0
    return score

# Main training loop
model = GPTLanguageModel()
model.load_state_dict(torch.load('model.pth', map_location='cpu'))
device = 'cpu'
model.eval()  # Set to evaluation mode

num_activations = 60
best_masks = [None] * 8  # Increased to 8 to match n_layer + 1
min_surprise = 100000
num_train_steps = 1000
num_layers = 8

with torch.no_grad():
    for i in range(num_train_steps):
        print(f"Training step {i+1}/{num_train_steps}")
        
        covariance_matrices = []
        empirical_means = []
        estimated_gaussians = []
        layer_masks = []
        input_points = []
        
        # Generate input points
        for iter in range(num_activations):
            xb, yb = get_batch('train')
            input_point = xb.to(device)
            input_points.append(input_point)
        
        current_layer_inputs = []
        for input_point in input_points:
            input_point = input_point.to(device).long()  # Ensure input is long type for embeddings
            with torch.no_grad():
                B, T = input_point.shape
                # Token embeddings
                tok_emb = model.token_embedding_table(input_point).float()  # (B, T, n_embd)
                # Position embeddings
                pos_emb = model.position_embedding_table(torch.arange(T, device=device)).float()  # (T, n_embd)
                # Combined embeddings
                x = tok_emb + pos_emb  # (B, T, n_embd)
                current_layer_inputs.append(x)
        
        # Process each layer
        for layer in range(n_layer):
            empirical_mean, covariance_matrix, layer_activations = propagate_distribution(current_layer_inputs, layer, model)
            empirical_means.append(empirical_mean)
            covariance_matrices.append(covariance_matrix)

            layer_mask = make_layer_mask(best_masks[layer], covariance_matrix, lr=0.1)
            layer_masks.append(layer_mask)
            
            estimated_layer_gaussian = create_gaussian_distribution(empirical_mean, covariance_matrix, layer_mask)
            estimated_gaussians.append(estimated_layer_gaussian)
            current_layer_inputs = sample_next_layer_inputs(
                estimated_layer_gaussian, layer_activations, num_activations
            )
                # sampled_activations = estimated_layer_gaussian.sample((num_activations,))  # Shape: (num_activations, feature_dim)
                # # Reshape to match expected input format (B, T, n_embd)
                # current_layer_inputs = [sampled_activations[i].unsqueeze(0).unsqueeze(0) for i in range(num_activations)]

        # Calculate surprise metrics
        layer_activations = []
        for sampled_activation in current_layer_inputs:
            sampled_activation = sampled_activation.to(device).float()  # Ensure float type
            with torch.no_grad():
                # Final layer norm
                ln_f_out = model.ln_f(sampled_activation)
                # Language model head (logits)
                logits = model.lm_head(ln_f_out)
                layer_activations.append(logits)
        empirical_mean, covariance_matrix = calculate_layer_statistics(layer_activations)
        dim = covariance_matrix.shape[0]
        layer_mask = torch.ones(dim, dim, dtype=torch.bool, device=covariance_matrix.device)
        estimated_layer_gaussian = create_efficient_gaussian_distribution(empirical_mean, covariance_matrix, layer_mask)

        estimated_output_distribution = estimated_layer_gaussian
        true_outputs, generated_tokens = find_true_outputs(model, input_points, device)
        
        surprise_estimate = surprise_of_estimate(true_outputs, estimated_output_distribution)
        surprise_metric1 = surprise_of_argument_metric1(layer_masks, covariance_matrices)
        print(surprise_estimate, surprise_metric1)
        print(generated_tokens)
        total_surprise = surprise_estimate + surprise_metric1
        print(f"Current total surprise: {total_surprise}")
        if total_surprise < min_surprise:
            min_surprise = total_surprise
            best_masks = layer_masks.copy()
            print(f"New best surprise: {min_surprise}")




# 1. Deploy on one example
# max_allowed_anomaly_score = 0
# for datapoint in dataset:
#     activation = record_activation(model, input)
#     anomaly_score = score_input(activation, best_heuristic_argument, covariance_matrices)
#     if max_allowed_anomaly_score < anomaly_score:
#         max_allowed_anomaly_score = anomaly_score
# input = 5
# activation = record_activation(model, input)
# anomaly_score = score_input(activation, best_heuristic_argument, covariance_matrices)
# if anomaly_score > max_allowed_anomaly_score:
#     print("ANOMALY")
# else:
#     print("ALL GOOD")


# 2. Estimate loss over one distribution 


# 3. Perform Low Probability Estimation over one distribution 


# 4. Fit only the last layer as a baseline

#205.51065063476562

Training step 1/1000
  Full dimension: 65, Active dimensions: 65
  Added regularization: 5.28e-07
  Successfully created 65-dimensional Gaussian
  Covariance eigenvalues: 1.00e-06 to 5.44e-05


AttributeError: 'tuple' object has no attribute 'log_prob'

In [ ]:
import torch
import random
from torch.distributions import MultivariateNormal
import matplotlib.pyplot as plt
import numpy as np

'''
Enhanced version with comprehensive monitoring and visualization
'''

def calculate_layer_statistics(activations):
    stacked_activations = []
    for activation in activations:
        # Ensure consistent dtype
        activation = activation.float()
        # Flatten batch and time dimensions: (B, T, n_embd) -> (B*T, n_embd)
        flattened = activation.view(-1, activation.shape[-1])
        stacked_activations.append(flattened)
    all_samples = torch.cat(stacked_activations, dim=0)
    
    print(f"  Statistics calculation: {len(activations)} activations -> {all_samples.shape[0]} total samples")
    print(f"  Feature dimension: {all_samples.shape[-1]}")
    
    # empirical mean μ̂ = (1/N) Σ(i=1 to N) x_i
    empirical_mean = torch.mean(all_samples, dim=0)
    # Center the data
    centered_samples = all_samples - empirical_mean.unsqueeze(0)
    # covariance matrix Σ̂ = (1/N) Σ(i=1 to N) (x_i - μ̂)(x_i - μ̂)^T
    covariance_matrix = torch.mm(centered_samples.t(), centered_samples) / (all_samples.shape[0] - 1)
    
    # Print statistics
    print(f"  Mean stats - min: {empirical_mean.min():.4f}, max: {empirical_mean.max():.4f}, norm: {empirical_mean.norm():.4f}")
    print(f"  Covariance stats - min: {covariance_matrix.min():.4f}, max: {covariance_matrix.max():.4f}")
    print(f"  Covariance diagonal - min: {torch.diag(covariance_matrix).min():.4f}, max: {torch.diag(covariance_matrix).max():.4f}")
    
    return empirical_mean, covariance_matrix

def create_sparse_argument_masks(covariance_matrix):
    dim = covariance_matrix.shape[0]
    # Create diagonal mask (to assume independence between activations)
    mask = torch.eye(dim, dtype=torch.bool, device=covariance_matrix.device)
    print(f"  Created diagonal mask: {dim}x{dim}, {torch.sum(mask)} True elements")
    return mask

def make_layer_mask(best_layer_mask, covariance_matrix, lr=0.05):
    if best_layer_mask is None:
        print("  No previous mask, creating diagonal mask")
        return create_sparse_argument_masks(covariance_matrix)
    
    new_mask = best_layer_mask.clone()
    dim = best_layer_mask.shape[0]
    original_true_count = torch.sum(new_mask).item()
    
    def is_positive_definite(matrix):
        try:
            torch.linalg.cholesky(matrix)
            return True
        except torch.linalg.LinAlgError:
            return False
    
    # Go through upper triangular positions
    positions = [(i, j) for i in range(dim) for j in range(i, dim)]
    random.shuffle(positions)
    flips_attempted = 0
    flips_accepted = 0
    
    for i, j in positions:
        # Skip with probability (1 - lr)
        if random.random() >= lr:
            continue
        flips_attempted += 1
        
        # Try flipping the bit
        original_val = new_mask[i, j].item()
        new_mask[i, j] = not original_val
        # Maintain symmetry
        if i != j:
            new_mask[j, i] = new_mask[i, j]
        # Test if the masked matrix is still positive definite
        masked_matrix = covariance_matrix * new_mask.float()
        if not is_positive_definite(masked_matrix):
            new_mask[i, j] = original_val
            if i != j: 
                new_mask[j, i] = original_val
        else:
            flips_accepted += 1
    
    final_true_count = torch.sum(new_mask).item()
    print(f"  Mask evolution: {original_true_count} -> {final_true_count} True elements")
    print(f"  Flip attempts: {flips_attempted}, accepted: {flips_accepted}")
    
    return new_mask

def get_activation(input_points, layer, model, device='cpu'):
    print(f"  Getting activations for layer {layer}")
    layer_activations = []
    block_idx = layer
    for i, sampled_activation in enumerate(input_points):
        sampled_activation = sampled_activation.to(device).float()  # Ensure float type
        with torch.no_grad():
            x = sampled_activation
            # Pass through the specific transformer block
            block = model.blocks[block_idx]
            # Pre-attention layer norm
            ln1_out = block.ln1(x)
            # Self-attention
            sa_out = block.sa(ln1_out)
            # Residual connection after attention
            x = x + sa_out
            # Pre-feedforward layer norm
            ln2_out = block.ln2(x)
            # Feedforward
            ffwd_out = block.ffwd(ln2_out)
            # Residual connection after feedforward
            x = x + ffwd_out
            layer_activations.append(x)
            
            if i == 0:  # Print stats for first activation
                print(f"    Input shape: {sampled_activation.shape}, output shape: {x.shape}")
                print(f"    Output stats - min: {x.min():.4f}, max: {x.max():.4f}, mean: {x.mean():.4f}")
    
    return layer_activations

def propagate_distribution(input_points, layer, model):
    print(f"\n--- Processing Layer {layer} ---")
    layer_activations = get_activation(input_points, layer, model)
    layer_empirical_mean, layer_covariance_matrix = calculate_layer_statistics(layer_activations)
    return layer_empirical_mean, layer_covariance_matrix, layer_activations

def create_gaussian_distribution(empirical_mean, covariance_matrix, layer_mask):
    # Only keep covariance terms where mask is True, set others to 0
    masked_cov = covariance_matrix * layer_mask.float()
    
    # Check if the masked covariance is positive definite
    try:
        distribution = MultivariateNormal(loc=empirical_mean, covariance_matrix=masked_cov)
        print(f"  Successfully created Gaussian distribution")
        print(f"  Mean norm: {empirical_mean.norm():.4f}")
        print(f"  Masked covariance eigenvalues range: {torch.linalg.eigvals(masked_cov).real.min():.6f} to {torch.linalg.eigvals(masked_cov).real.max():.6f}")
        return distribution
    except Exception as e:
        print(f"  Error creating Gaussian: {e}")
        # Fallback to diagonal covariance
        diag_cov = torch.diag(torch.diag(masked_cov))
        distribution = MultivariateNormal(loc=empirical_mean, covariance_matrix=diag_cov)
        print(f"  Fallback to diagonal covariance")
        return distribution

def sample_next_layer_inputs(estimated_layer_gaussian, original_activations, num_activations):
    # Get the original shape from one of the activations
    sample_activation = original_activations[0]
    batch_size, seq_len, feature_dim = sample_activation.shape
    
    print(f"  Sampling {num_activations} activations with shape ({batch_size}, {seq_len}, {feature_dim})")
    
    # Sample from the distribution
    # We need (num_activations * batch_size * seq_len) samples
    total_samples_needed = num_activations * batch_size * seq_len
    try:
        sampled_flat = estimated_layer_gaussian.sample((total_samples_needed,))
        print(f"  Successfully sampled {total_samples_needed} points")
        print(f"  Sample stats - min: {sampled_flat.min():.4f}, max: {sampled_flat.max():.4f}, mean: {sampled_flat.mean():.4f}")
    except Exception as e:
        print(f"  Sampling error: {e}")
        # Fallback to mean values
        sampled_flat = estimated_layer_gaussian.mean.unsqueeze(0).repeat(total_samples_needed, 1)
        print(f"  Fallback to mean values")
    
    # Reshape to match the expected format: (num_activations, batch_size, seq_len, feature_dim)
    sampled_activations = sampled_flat.view(num_activations, batch_size, seq_len, feature_dim)
    
    # Convert back to list format expected by the rest of the code
    current_layer_inputs = [sampled_activations[i] for i in range(num_activations)]
    
    return current_layer_inputs

def find_true_outputs(model, input_points, device='cpu'):
    print("\n--- Finding True Outputs ---")
    true_outputs = []
    generated_tokens = []  # Store the actual tokens generated
    
    with torch.no_grad():
        for i, input_point in enumerate(input_points):
            input_point = input_point.to(device).long()  # Ensure long type for embeddings
            B, T = input_point.shape
            
            # Token embeddings
            tok_emb = model.token_embedding_table(input_point).float()  # (B, T, n_embd)
            
            # Position embeddings
            pos_emb = model.position_embedding_table(torch.arange(T, device=device)).float()  # (T, n_embd)
            
            # Combined embeddings
            x = tok_emb + pos_emb  # (B, T, n_embd)
            
            # Pass through all transformer blocks
            for block in model.blocks:
                # Pre-attention layer norm
                ln1_out = block.ln1(x)
                # Self-attention
                sa_out = block.sa(ln1_out)
                # Residual connection after attention
                x = x + sa_out
                # Pre-feedforward layer norm
                ln2_out = block.ln2(x)
                # Feedforward
                ffwd_out = block.ffwd(ln2_out)
                # Residual connection after feedforward
                x = x + ffwd_out
            
            # Final layer norm
            ln_f_out = model.ln_f(x)  # (B, T, n_embd)
            
            # Language model head (logits)
            logits = model.lm_head(ln_f_out)  # (B, T, vocab_size)
            
            if i == 0:  # Print stats for first input
                print(f"  Input shape: {input_point.shape}")
                print(f"  Final logits shape: {logits.shape}")
                print(f"  Logits stats - min: {logits.min():.4f}, max: {logits.max():.4f}")
            
            # Convert logits to actual tokens
            # Take the last token prediction for each sequence
            last_token_logits = logits[:, -1, :]  # (B, vocab_size)
            predicted_token_ids = torch.argmax(last_token_logits, dim=-1)  # (B,)
            
            # Decode to actual text
            for batch_idx in range(B):
                token_id = predicted_token_ids[batch_idx].item()
                predicted_char = itos[token_id]  # Convert to character using your decoder
                generated_tokens.append(predicted_char)
            
            # Flatten the logits to match expected input format for calculate_layer_statistics
            # This converts (B, T, vocab_size) -> (B*T, vocab_size)
            flattened_logits = logits.view(-1, logits.shape[-1])
            true_outputs.append(flattened_logits)
    
    print(f"  Generated {len(generated_tokens)} tokens: {''.join(generated_tokens[:20])}...")
    return true_outputs, generated_tokens

def surprise_of_estimate(true_samples, estimated_output_distribution):
    print("\n--- Computing Surprise of Estimate ---")
    total_log_prob = 0.0
    sample_count = 0
    
    for i, sample in enumerate(true_samples):
        # Process each sample individually
        for j in range(sample.shape[0]):
            single_sample = sample[j]  # Single vector of vocab_size
            try:
                log_prob = estimated_output_distribution.log_prob(single_sample)
                total_log_prob += log_prob
                sample_count += 1
                
                if i == 0 and j == 0:  # Print stats for first sample
                    print(f"  First sample log prob: {log_prob:.4f}")
                    print(f"  Sample stats - min: {single_sample.min():.4f}, max: {single_sample.max():.4f}")
            except Exception as e:
                print(f"  Error computing log prob for sample {i},{j}: {e}")
                # Use a large negative log prob as penalty
                total_log_prob += -1000.0
                sample_count += 1
    
    surprise = -(total_log_prob / sample_count).item()
    print(f"  Total samples processed: {sample_count}")
    print(f"  Average log prob: {total_log_prob / sample_count:.4f}")
    print(f"  Surprise (negative log likelihood): {surprise:.4f}")
    return surprise

def surprise_of_argument_metric1(layer_masks, covariance_matrices):
    print("\n--- Computing Argument Surprise Metric 1 ---")
    total_surprise = 0.0
    for i, (mask, cov_matrix) in enumerate(zip(layer_masks, covariance_matrices)):
        # Count number of True elements in the mask (|πᵢ|)
        num_tracked_elements = torch.sum(mask.float()).item()
        # Dimension of the space (dim(Xᵢ))
        dimension = cov_matrix.shape[0]
        # Surprise is the excess over diagonal elements
        layer_surprise = num_tracked_elements - dimension
        layer_surprise = max(0, layer_surprise)
        total_surprise += layer_surprise
        print(f"  Layer {i}: {num_tracked_elements}/{dimension**2} elements tracked, surprise: {layer_surprise}")
    
    print(f"  Total argument surprise: {total_surprise}")
    return total_surprise

def create_visualization_plots(step, surprise_estimates, surprise_metrics, total_surprises, best_surprises):
    """Create plots to visualize the training progress"""
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(12, 8))
    
    steps = range(1, len(surprise_estimates) + 1)
    
    # Plot surprise estimates
    ax1.plot(steps, surprise_estimates, 'b-', label='Surprise Estimate')
    ax1.set_title('Surprise of Estimate Over Time')
    ax1.set_xlabel('Training Step')
    ax1.set_ylabel('Surprise')
    ax1.grid(True)
    ax1.legend()
    
    # Plot surprise metrics
    ax2.plot(steps, surprise_metrics, 'r-', label='Argument Surprise')
    ax2.set_title('Surprise of Argument Over Time')
    ax2.set_xlabel('Training Step')
    ax2.set_ylabel('Surprise')
    ax2.grid(True)
    ax2.legend()
    
    # Plot total surprises
    ax3.plot(steps, total_surprises, 'g-', label='Total Surprise')
    ax3.plot(steps, best_surprises, 'g--', label='Best So Far')
    ax3.set_title('Total Surprise Over Time')
    ax3.set_xlabel('Training Step')
    ax3.set_ylabel('Total Surprise')
    ax3.grid(True)
    ax3.legend()
    
    # Plot ratio
    ratios = [est/met if met > 0 else 0 for est, met in zip(surprise_estimates, surprise_metrics)]
    ax4.plot(steps, ratios, 'm-', label='Estimate/Argument Ratio')
    ax4.set_title('Surprise Ratio Over Time')
    ax4.set_xlabel('Training Step')
    ax4.set_ylabel('Ratio')
    ax4.grid(True)
    ax4.legend()
    
    plt.tight_layout()
    plt.savefig(f'training_progress_step_{step}.png', dpi=150, bbox_inches='tight')
    plt.close()

# Load your model and other necessary components
# (assuming these are defined from your first file)
# model = GPTLanguageModel()
# model.load_state_dict(torch.load('model.pth', map_location='cpu'))
# device = 'cpu'
# model = model.to(device)
# model.eval()

# Training tracking variables
surprise_estimates_history = []
surprise_metrics_history = []
total_surprises_history = []
best_surprises_history = []

# Main training loop with enhanced monitoring
print("=" * 60)
print("STARTING ENHANCED TRAINING WITH MONITORING")
print("=" * 60)

num_activations = 600
best_masks = [None] * 8  # Increased to 8 to match n_layer + 1
min_surprise = 100000
num_train_steps = 1000
num_layers = 8

with torch.no_grad():
    for i in range(num_train_steps):
        print(f"\n{'='*20} Training Step {i+1}/{num_train_steps} {'='*20}")
        
        covariance_matrices = []
        empirical_means = []
        estimated_gaussians = []
        layer_masks = []
        input_points = []
        
        # Generate input points
        print(f"\n--- Generating {num_activations} Input Points ---")
        for iter in range(num_activations):
            xb, yb = get_batch('train')
            input_point = xb.to(device)
            input_points.append(input_point)
        print(f"Generated input points with shape: {input_points[0].shape}")
        
        # Initialize current layer inputs (embeddings)
        print("\n--- Computing Initial Embeddings ---")
        current_layer_inputs = []
        for input_point in input_points:
            input_point = input_point.to(device).long()  # Ensure input is long type for embeddings
            with torch.no_grad():
                B, T = input_point.shape
                # Token embeddings
                tok_emb = model.token_embedding_table(input_point).float()  # (B, T, n_embd)
                # Position embeddings
                pos_emb = model.position_embedding_table(torch.arange(T, device=device)).float()  # (T, n_embd)
                # Combined embeddings
                x = tok_emb + pos_emb  # (B, T, n_embd)
                current_layer_inputs.append(x)
        print(f"Initial embeddings shape: {current_layer_inputs[0].shape}")
        
        # Process each layer
        for layer in range(n_layer):
            empirical_mean, covariance_matrix, layer_activations = propagate_distribution(current_layer_inputs, layer, model)
            empirical_means.append(empirical_mean)
            covariance_matrices.append(covariance_matrix)

            layer_mask = make_layer_mask(best_masks[layer], covariance_matrix, lr=0.1)
            layer_masks.append(layer_mask)
            
            estimated_layer_gaussian = create_gaussian_distribution(empirical_mean, covariance_matrix, layer_mask)
            estimated_gaussians.append(estimated_layer_gaussian)
            current_layer_inputs = sample_next_layer_inputs(
                estimated_layer_gaussian, layer_activations, num_activations
            )

        # Final layer processing
        print("\n--- Processing Final Layer (Output) ---")
        layer_activations = []
        for sampled_activation in current_layer_inputs:
            sampled_activation = sampled_activation.to(device).float()  # Ensure float type
            with torch.no_grad():
                # Final layer norm
                ln_f_out = model.ln_f(sampled_activation)
                # Language model head (logits)
                logits = model.lm_head(ln_f_out)
                layer_activations.append(logits)
        
        empirical_mean, covariance_matrix = calculate_layer_statistics(layer_activations)
        dim = covariance_matrix.shape[0]
        layer_mask = torch.ones(dim, dim, dtype=torch.bool, device=covariance_matrix.device)
        estimated_layer_gaussian = create_gaussian_distribution(empirical_mean, covariance_matrix, layer_mask)

        estimated_output_distribution = estimated_layer_gaussian
        true_outputs, generated_tokens = find_true_outputs(model, input_points, device)
        
        # Calculate surprises
        surprise_estimate = surprise_of_estimate(true_outputs, estimated_output_distribution)
        surprise_metric1 = surprise_of_argument_metric1(layer_masks, covariance_matrices)
        total_surprise = surprise_estimate + surprise_metric1
        
        # Update tracking
        surprise_estimates_history.append(surprise_estimate)
        surprise_metrics_history.append(surprise_metric1)
        total_surprises_history.append(total_surprise)
        best_surprises_history.append(min_surprise)
        
        # Print summary
        print(f"\n{'='*15} STEP {i+1} SUMMARY {'='*15}")
        print(f"Surprise Estimate: {surprise_estimate:.4f}")
        print(f"Argument Surprise: {surprise_metric1:.4f}")
        print(f"Total Surprise: {total_surprise:.4f}")
        print(f"Best So Far: {min_surprise:.4f}")
        print(f"Sample Generated Tokens: {''.join(generated_tokens[:50])}")
        
        # Update best masks if needed
        if total_surprise < min_surprise:
            min_surprise = total_surprise
            best_masks = layer_masks.copy()
            print(f"🎉 NEW BEST SURPRISE: {min_surprise:.4f}")
        
        # Create visualization every 10 steps
        if (i + 1) % 10 == 0:
            create_visualization_plots(i + 1, surprise_estimates_history, 
                                     surprise_metrics_history, total_surprises_history, 
                                     best_surprises_history)
            print(f"Saved visualization plot: training_progress_step_{i+1}.png")

print(f"\n{'='*20} TRAINING COMPLETE {'='*20}")
print(f"Final best surprise: {min_surprise:.4f}")
print(f"Final surprise estimate: {surprise_estimates_history[-1]:.4f}")
print(f"Final argument surprise: {surprise_metrics_history[-1]:.4f}")

STARTING ENHANCED TRAINING WITH MONITORING

==================== Training Step 1/1000 ====================

--- Generating 60 Input Points ---
Generated input points with shape: torch.Size([64, 256])

--- Computing Initial Embeddings ---
Initial embeddings shape: torch.Size([64, 256, 384])

--- Processing Layer 0 ---
  Getting activations for layer 0
    Input shape: torch.Size([64, 256, 384]), output shape: torch.Size([64, 256, 384])
    Output stats - min: -3.1637, max: 4.2184, mean: 0.0014
  Statistics calculation: 60 activations -> 983040 total samples
  Feature dimension: 384
  Mean stats - min: -0.2104, max: 0.3011, norm: 1.3428
  Covariance stats - min: -0.0859, max: 0.1895
  Covariance diagonal - min: 0.0654, max: 0.1895
  No previous mask, creating diagonal mask
  Created diagonal mask: 384x384, 384 True elements
  Successfully created Gaussian distribution
  Mean norm: 1.3428
  Masked covariance eigenvalues range: 0.065439 to 0.189489
  Sampling 60 activations with shape (64,

KeyboardInterrupt: 